In [1]:
#!pip install tensorflow
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten
from keras.layers import Conv1D
from keras.models import Sequential

from keras.layers import LSTM
import itertools
import matplotlib.pyplot as plt
%matplotlib inline



C:\local\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load test data

In [2]:
data=np.load('overnight_all_compressed.npz')
pred_data = data['arr_2']

Load train data from zip file and merge all files in to single array

In [3]:
x = np.load('overnight_all_compressed_new.npz')
print(x)
x_t=[]
final_array=np.array([])

a = x['arr_0']
b = x['arr_1']

print(a.shape)
print(b.shape)


final_array=np.vstack([a,b])

print(final_array.shape)

'''for i in x:
    
    print(x[i].shape)
    temp=x[i]
    final_array = np.hstack()
    final_array = np.concatenate((final_array,temp),axis=0)'''
    #np.concatenate(p,x[i])
    #p =np.concatenate((p,temp),axis=0)
    #np.append(x[i])
    
#data=['seconds','mic (trachea)','abdominal movement (stretch sensor)','thorax movement (stretch sensor)']
#x = pd.DataFrame(x['arr_0'], columns = data)
#x = pd.DataFrame(x['arr_1'], columns = data)

#merged = list(itertools.chain.from_iterable(x_t))


(11044514, 4)
(4769867, 4)
(15814381, 4)


'for i in x:\n    \n    print(x[i].shape)\n    temp=x[i]\n    final_array = np.hstack()\n    final_array = np.concatenate((final_array,temp),axis=0)'

Divide data into X & Y sets for the train and validation data

In [4]:
X_train = final_array[:,1]
Y_train = final_array[:,2]

Divide X & Y sets for the test data

In [5]:
time=pred_data[:100000,0]
x_pred=pred_data[:100000,5].reshape(100000,1,1)
y_true=pred_data[:100000,2]

data manipulation for time series

In [6]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


model 2 with 1D Convolution layers and final dense/FC layer with filter of size 3, 1, 1

In [7]:
reframed = series_to_supervised(final_array, 1, 1)

In [8]:
reframed

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var1(t),var2(t),var3(t),var4(t)
1,0.000,1.279312,2.073506,1.995487,0.001,1.215297,2.075507,1.997488
2,0.001,1.215297,2.075507,1.997488,0.002,1.243304,2.073506,1.997488
3,0.002,1.243304,2.073506,1.997488,0.003,1.333326,2.071506,1.993487
4,0.003,1.333326,2.071506,1.993487,0.004,1.243304,2.073506,1.997488
5,0.004,1.243304,2.073506,1.997488,0.005,1.289315,2.073506,1.995487
6,0.005,1.289315,2.073506,1.995487,0.006,1.233301,2.071506,1.993487
7,0.006,1.233301,2.071506,1.993487,0.007,1.271310,2.069505,1.991486
8,0.007,1.271310,2.069505,1.991486,0.008,1.267309,2.073506,1.995487
9,0.008,1.267309,2.073506,1.995487,0.009,1.209295,2.073506,1.997488
10,0.009,1.209295,2.073506,1.997488,0.010,1.259308,2.073506,1.995487


In [9]:
reframed.drop(reframed.columns[[0,3,4,5,7]], axis =1, inplace=True)

In [11]:
#ytest

manually splitting train data  into train (70%) and validation (30% )

In [12]:
l= len(reframed)
Xtrain = reframed[:round(l*0.7)]
Xtest = reframed[round(l*0.7)+1:]
ytrain = reframed[:round(l*0.7)]
ytest = reframed[round(l*0.7)+1:]

In [13]:
Xtrain.drop(Xtrain.columns[[2]], axis =1, inplace=True)
Xtest.drop(Xtest.columns[[2]], axis =1, inplace=True)
ytrain.drop(ytrain.columns[[0,1]], axis =1, inplace=True)
ytest.drop(ytest.columns[[0,1]], axis =1, inplace=True)

C:\local\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\local\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\local\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\local\Anaconda3\lib\site-packages\

In [14]:
Xtrain=np.array(Xtrain)
ytrain=np.array(ytrain)
Xtest=np.array(Xtest)
ytest=np.array(ytest)

In [17]:
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(11070066, 1, 2)
(11070066, 1)
(4744313, 1, 2)
(4744313, 1)


In [16]:
Xtrain=(Xtrain).reshape((Xtrain.shape[0], 1, Xtrain.shape[1]))

#ytrain=ytrain.reshape((ytrain.shape[0], 1, ytrain.shape[1]))

Xtest=Xtest.reshape((Xtest.shape[0], 1, Xtest.shape[1]))

#ytest=ytest.reshape((ytest.shape[0], 1, ytest.shape[1]))

validationData=(Xtest,ytest)

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model = Sequential()
model.add(LSTM(50, input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(Xtrain, ytrain, epochs=10, batch_size=7000, validation_data=validationData, verbose=2, shuffle=False)
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()


Train on 11070066 samples, validate on 4744313 samples
Epoch 1/10
 - 73s - loss: 0.0849 - val_loss: 0.0831
Epoch 2/10
 - 72s - loss: 0.0201 - val_loss: 0.0362
Epoch 3/10
 - 72s - loss: 0.0090 - val_loss: 0.0043
Epoch 4/10
 - 72s - loss: 0.0057 - val_loss: 0.0042
Epoch 5/10
 - 71s - loss: 0.0050 - val_loss: 0.0060
Epoch 6/10
 - 71s - loss: 0.0042 - val_loss: 0.0045
Epoch 7/10
 - 71s - loss: 0.0038 - val_loss: 0.0034
Epoch 8/10
 - 72s - loss: 0.0033 - val_loss: 0.0032
Epoch 9/10
 - 72s - loss: 0.0032 - val_loss: 0.0032
Epoch 10/10


In [ ]:

# make a prediction
yhat = model.predict(Xtest)
Xtest = Xtest.reshape((Xtest.shape[0], Xtest.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, Xtest[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
ytest = ytest.reshape((len(test_y), 1))
inv_y = concatenate((ytest, Xtest[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Source https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/